# Try this one out.

In [ ]:
import torch

import numpy as np
import matplotlib.pyplot as plt

import helper

%matplotlib inline

Here we'll use the Fashion MNIST dataset which consists of greyscale clothes and shoes type. Each image is 28x28 pixels, you can see a sample below

<img src='images/fashion-mnist-sprite.png'>

Our goal is to build a neural network that can take one of these images and predict the clothes or shoes type in the image.

Just like before, we need to get our dataset. This is also provided through the `torchvision` package. The code below will download the FashionMNIST dataset, then create training and test datasets for us.

In [ ]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/Fashion_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

plt.imshow(images[1].numpy().squeeze(), cmap='Greys_r');

## Building networks with PyTorch

Here I want you to try build and train a network, just like the MNIST Handwritten one. You should use 784 inputs, 256 hidden units, 10 output units and a log_softmax output. Use ReLU as activation function where suitable.

I've created a template for to use below, but you have an option of using nn.Sequential module.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Inputs to hidden layer linear transformation
        self.hidden = 
        # Output layer, 10 units - one for each digit
        self.output =
        
    def forward(self, x):
        # Pass the input tensor through each of our operations
        
        return x

In [ ]:
model = Net()
model

In [ ]:
import torch.optim as optim

In [ ]:
# define the criterion and optimizer
criterion = 
optimizer =

## Training the network

In [ ]:
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
    
        # TODO: Training pass
        
        loss = 
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

In [ ]:
images, labels = next(iter(trainloader))

img = images[0].view(1, 784)
# Turn off gradients to speed up this part
with torch.no_grad():
    logps = model(img)

# Output of the network are log-probabilities, need to take exponential for probabilities
ps = torch.exp(logps)
helper.view_classify(img.view(1, 28, 28), ps)

# Save the Trained Model

In [ ]:
# Save your mode
